In [ ]:
from smolagents import CodeAgent, LiteLLMModel

In [ ]:
model = LiteLLMModel(
    model_id="ollama/qwen2.5:7b-instruct",
    api_base="http://localhost:11434",
    system_prompt="You are a terse Python coding assistant. Use tools precisely."
)

In [ ]:
agent = CodeAgent(
    model=model,
    tools=[],
    additional_authorized_imports=["duckdb", "pandas", "csv"],
)

In [ ]:
prompt = """
You are a data analyst. Use duckdb to analyze the following CSV data.
Your job is to answer the questions asked by the user about the given dataset.
Only answer to that question and don't try to do any other analysis and don't try to explain any other thing.

The file is located in the current working directory.

## Data dictionary

1. ts: timestamp
2. step: step type, measurement or interraction
3. feature: feature name of the measurement, OCP, CA, CV
4. sample: number of the measured sample
5. value_ocp: result value of the OCP measurement
6. value_ca: result value of the CA measurement
7. value_cv: result value of the CV measurement
8. t_start: start time of the step
9. t_end: end time of the step
10. duration_s: duration of the step in seconds
11. metrics: metrics of the step, e.g. current, voltage, capacity, energy
12. meta: meta information of the step, e.g. cycle index, step index, sample index

* You can use duckdb to load, query and analyze the CSV data.

FEW SHOT EXAMPLES
1. How many rows are in the CSV file?
ANSWER: There are 1000 rows in the CSV file.
2. What is the average OCP value for all measurements?
ANSWER: The average OCP value for all measurements is 0.8V
3. What is the maximum CA value for all measurements?
ANSWER: The maximum CA value for all measurements is 1.2A
"""


In [ ]:
question = """What is the average value_ocp for each feature in the dataset?"""
file = r"C:\Users\chekim\Workspace\llm_robotic_control\experiment_logs\20250924104104.csv"

response = agent.run(
    prompt + "\n" + question, 
    additional_args = dict(source_file=file) 
)

print(response)